In [ ]:
import re
import os
from dataclasses import dataclass
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

In [ ]:
pg_conn_string = os.getenv("PG_CONN_STRING", "dbname=wrenchball user=postgres password=postgres host=localhost port=5932")

In [ ]:
@dataclass
class User():
    name: str
    initial: str
    discord_snowflake: str
    role: str

In [ ]:
users = [
    User(name="Andrew", initial="AL", discord_snowflake="194910567188791297", role="admin"),
    User(name="Jack", initial="JH", discord_snowflake="178958252820791296", role="admin"),
    User(name="Michael", initial="MC", discord_snowflake="171102907079327745", role="admin"),
    User(name="Matthew", initial="MR", discord_snowflake="687449838945828925", role="user"),
    User(name="ADwy", initial="AD", discord_snowflake="200403036252798976", role="user"),
    User(name="NDA", initial="NDA", discord_snowflake="89554070678827008", role="user"),
    User(name="Evan", initial="EO", discord_snowflake="478366445995032596", role="user"),
    User(name="Luke", initial="LB", discord_snowflake="158029629683269632", role="user"),
    User(name="Noran", initial="AN", discord_snowflake="262332170041819146", role="user"),
    User(name="Adam", initial="AW", discord_snowflake="606620386225946624", role="user"),
]

In [ ]:
import psycopg

with psycopg.connect(pg_conn_string) as conn:
    with conn.cursor() as cur:
        for user in users:
            cur.execute(
                """
                INSERT INTO "user" (name, role, discord_snowflake)
                VALUES (%s, %s, %s)
                """,
                (user.name, user.role, user.discord_snowflake)
            )

    conn.commit()

In [ ]:
with psycopg.connect(pg_conn_string) as conn:
    with conn.cursor() as cur:
        for user in users:
            # Fetch user id from the database using discord_snowflake
            cur.execute(
                """
                SELECT id FROM "user" WHERE discord_snowflake = %s
                """,
                (user.discord_snowflake,)
            )
            result = cur.fetchone()
            if result is not None:
                user_id = result[0]
                team_name = f"{user.name}'s Team"
                abbreviation = user.initial
                cur.execute(
                    """
                    INSERT INTO team (name, user_id, abbreviation, color)
                    VALUES (%s, %s, %s, 'white')
                    """,
                    (team_name, user_id, abbreviation)
                )
            else:
                raise ValueError(f"User {user.name} not found in database")
    conn.commit()

In [ ]:
s3_xlsx_path = Path("data/sheets/lil sLUg crew S3.xlsx")

s3_df = pd.read_excel(s3_xlsx_path, sheet_name="Character List")

s3_df.head()

In [ ]:
players = [c for c in s3_df["Character"].dropna().unique().tolist() if c != 'Empty']

players

In [ ]:
# https://docs.google.com/spreadsheets/d/13uVzRaigr4i-QRZAhp4DcbDuhxhJvwxZigWds_Y8CR4/edit?gid=66049872#gid=66049872

stat_table_xlsx_path = Path("data/sheets/Sluggers Stat Table.xlsx")

stat_table_df = pd.read_excel(stat_table_xlsx_path, sheet_name="Relevant Stats", header=2)
stat_table_df = stat_table_df[~stat_table_df["Character"].astype(str).str.startswith("Unused")]
stat_table_df["Captain"] = stat_table_df["Captain"].apply(lambda x: True if str(x).strip().lower() == "yes" else False)
stat_table_df["Stamina"] = stat_table_df["Stamina"].astype(str).str.replace(",", "").astype(int)

def cleanup_character_name(name):
    cleaned_name = name
    if "Koopa Troopa" in str(cleaned_name):
        cleaned_name = cleaned_name.replace("Koopa Troopa", "Koopa")

    if "Koopa Paratroopa" in str(cleaned_name):
        cleaned_name = cleaned_name.replace("Koopa Paratroopa", "Paratroopa")

    if str(cleaned_name).endswith(".") and not str(cleaned_name).endswith("Jr."):
        cleaned_name = cleaned_name[:-1]

    # Match pattern: <Base> (<Thing>)
    m = re.match(r"^(.*) \(([^)]+)\)$", str(cleaned_name))
    if m:
        base, thing = m.groups()
        if base == "Dark Bones":
            cleaned_name = base
        elif thing != "F":
            cleaned_name = f"{thing} {base}"

    return cleaned_name

stat_table_df["Character"] = stat_table_df["Character"].apply(cleanup_character_name)

stat_table_df.columns

In [ ]:
column_mapping = {
    "Character": "character",
    "Character Class": "character_class",
    "Captain": "captain",
    "Throwing Arm": "throwing_arm",
    "Batting Stance": "batting_stance",
    "Ability": "ability",
    "Weight": "weight",
    "Hitting\nTrajectory\n(sweet spot)": "hitting_trajectory",
    "Slap hit\ncontact size": "slap_hit_contact_size",
    "Charge Hit \nContact Size": "charge_hit_contact_size",
    "Slap Hit \nPower?": "slap_hit_power",
    "Charge Hit \nPower": "charge_hit_power",
    "Bunting": "bunting",
    "Speed": "speed",
    "Throwing\nSpeed": "throwing_speed",
    "Fielding": "fielding",
    "Curveball \nSpeed": "curveball_speed",
    "Fastball\nSpeed": "fastball_speed",
    "Curve": "curve",
    "Stamina": "stamina",
    "Pitching CSS": "pitching_css",
    "Batting CSS": "batting_css",
    "Fielding CSS": "fielding_css",
    "Speed CSS": "speed_css",
}

stat_table_df = stat_table_df.rename(columns=column_mapping)
stat_table_df = stat_table_df[list(column_mapping.values())]
stat_table_df.head(20)

In [ ]:
# find the overlap between players and the characters in the stat table
characters = stat_table_df["character"].unique().tolist()

overlap = set(players) & set(characters)

only_in_players = set(players) - set(characters)

only_in_characters = set(characters) - set(players)

list(only_in_characters)

In [ ]:
with psycopg.connect(pg_conn_string) as conn:
    with conn.cursor() as cur:
        for index, row in stat_table_df.iterrows():
            cur.execute(
                """
                INSERT INTO stat (character, character_class, captain, throwing_arm, batting_stance, ability, weight, hitting_trajectory, slap_hit_contact_size, charge_hit_contact_size, slap_hit_power, charge_hit_power, bunting, speed, throwing_speed, fielding, curveball_speed, fastball_speed, curve, stamina, pitching_css, batting_css, fielding_css, speed_css)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """,
                tuple(row)
            )

        conn.commit()

In [ ]:
mii_metadata = Path("data/miis/mii_metadata.csv")

mii_metadata_df = pd.read_csv(mii_metadata)

mii_metadata_df.head()

In [ ]:
import psycopg

dry_run = False

with psycopg.connect(pg_conn_string) as conn:
    with conn.cursor() as cur:
        for player_name in players:
            stats_character = player_name if player_name in stat_table_df["character"].values else None

            if stats_character is None:
                mii_name = player_name

                if mii_name == "Crelulu":
                    mii_name = "crululu"
                if mii_name == "The Grabber":
                    mii_name = "grabber"

                mii_row = mii_metadata_df[mii_metadata_df["mii_name"] == mii_name]
                if mii_row.empty:
                    mii_row = mii_metadata_df[mii_metadata_df["mii_name"] == mii_name.lower()]

                if not mii_row.empty:
                    favorite_color = mii_row.iloc[0]["favorite_color"]
                    if favorite_color == "DarkGreen":
                        favorite_color = "Green"
                    favorite_color = re.sub(r'([a-z])([A-Z])', r'\1 \2', favorite_color)
                    gender = mii_row.iloc[0]["gender"]
                    if gender == "Female":
                        stats_character_candidate = f"{favorite_color} Mii (F)"
                    else:
                        stats_character_candidate = f"{favorite_color} Mii"
                    if stats_character_candidate in stat_table_df["character"].values:
                        stats_character = stats_character_candidate
                    else:
                        raise ValueError(f"No stats character found for {player_name}, {mii_name}, {favorite_color}, {gender}")
                else:
                    raise ValueError(f"No mii metadata found for {player_name}")

            image_dir = Path("data/sluggerpics/sideview/right")
            name = player_name
            name_map = {
                "Shy Guy": "Red Shy Guy",
                "Koopa": "Green Koopa",
                "Paratroopa": "Red Paratroopa",
                "Magikoopa": "Red Magikoopa",
                "Dark Bones": "Gray Dry Bones",
            }
            if player_name in name_map:
                name = name_map[player_name]

            name = name.lower().replace('.', '')
            image_filename = f"{name}.png"
            image_path = image_dir / image_filename

            if image_path.is_file():
                image_url = f"/images/players/sideview/right/{image_filename}"
            else:
                image_dir_miis = Path("website/public/images/miis")
                image_filename_miis = f"{player_name.lower()}.png"
                image_path_miis = image_dir_miis / image_filename_miis

                if image_path_miis.is_file():
                    image_url = f"/images/miis/{image_filename_miis}"
                else:
                    print(f"Image not found for {player_name}")

            cur.execute(
                """
                INSERT INTO players (name, team_id, image_url, stats_character)
                VALUES (%s, %s, %s, %s)
                """,
                (player_name, None, image_url, stats_character)
            )
        if dry_run:
            conn.rollback()
            print("Dry run: rolled back all inserts.")
        else:
            conn.commit()
            print("Committed all inserts.")
